Connect to Snowflake for the IMDB datasets

In [1]:
from snowflake.snowpark.session import Session
from snowflake.ml.utils.connection_params import SnowflakeLoginOptions

pars = SnowflakeLoginOptions("test_conn")
pars["database"] = "IMDB"
session = Session.builder.configs(pars).create()
session.query_tag = "sentiment-1"

SnowflakeLoginOptions() is in private preview since 0.2.0. Do not use it in production. 


Isolate all code into a Python function

In [2]:
def train_imdb(session: Session, train_dataset_name: str, tmp_folder: str):
    
    from snowflake.snowpark import functions as fn
    import sklearn.feature_extraction.text as txt
    from sklearn import svm
    from joblib import dump
    
    df = session.table(train_dataset_name)
    df_flag = df.withColumn("SENTIMENT_FLAG",
        fn.when(df.SENTIMENT == "positive", 1).otherwise(2))
    train_x = df_flag.toPandas().REVIEW.values
    train_y = df_flag.toPandas().SENTIMENT_FLAG.values
    df_flag.show()

    filename = f'{tmp_folder}vect_review2.joblib'
    print(f'Building Sparse Matrix into {filename}...')
    vector = txt.CountVectorizer(
        token_pattern="[\\w']+\\w\\b", ngram_range=(1, 2), analyzer='word', 
        max_df=0.02, min_df=1 * 1./len(train_x), vocabulary=None, binary=True)
    bow = vector.fit_transform(train_x)
    dump(vector, filename, compress=True)
    session.file.put(filename, "@models", auto_compress=True, overwrite=True)

    filename = f'{tmp_folder}model_review2.joblib'
    print(f'Fitting model into {filename}...')
    model = svm.LinearSVC(C=1.8, max_iter=100)
    model.fit(bow, train_y)
    dump(model, filename, compress=True)
    session.file.put(filename, "@models", auto_compress=True, overwrite=True)

    return { "STATUS": "SUCCESS", "R2 Score Train": str(model.score(bow, train_y)) }

Call local Python function and list @MODELS stage files

In [3]:
ret = train_imdb(session, "TRAIN_DATASET", "../../../.spool/")
print(ret)

session.sql("LS @MODELS").show()

---------------------------------------------------------------------------------------
|"REVIEW"                                            |"SENTIMENT"  |"SENTIMENT_FLAG"  |
---------------------------------------------------------------------------------------
|"In Victorian times a father is separated from ...  |positive     |1                 |
|"When this film gets it right it really gets it...  |positive     |1                 |
|"What this film has is its realism , you really...  |negative     |2                 |
|"""Like the first touch of pleasure and guilt, ...  |positive     |1                 |
|"This is a weird and compelling film. The topic...  |positive     |1                 |
|"Although I am not a Michael Jackson fan, I lik...  |positive     |1                 |
|"Tigerland follows the lives of a group of rece...  |positive     |1                 |
|"To all the reviewers on this page, I would hav...  |positive     |1                 |
|"I really should give this stin

c:\Projects\snowflake-cortex\venv\lib\site-packages\sklearn\svm\_classes.py:32: FutureWarning: The default value of `dual` will change from `True` to `'auto'` in 1.5. Set the value of `dual` explicitly to suppress the warning.
  warnings.warn(
c:\Projects\snowflake-cortex\venv\lib\site-packages\sklearn\svm\_base.py:1250: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  warnings.warn(


----------------------------------------------------------------------------------------------------------------
|"name"                          |"size"    |"md5"                             |"last_modified"                |
----------------------------------------------------------------------------------------------------------------
|models/model_review.joblib.gz   |10820048  |1c63425ac807b5048c1e1f7ddc72da23  |Wed, 24 Apr 2024 15:52:14 GMT  |
|models/model_review1.joblib.gz  |10820800  |38506eb60f34f5d16a4c9d68384ef3ee  |Wed, 24 Apr 2024 18:08:37 GMT  |
|models/model_review2.joblib.gz  |10831088  |f4f7de7178e43463789bf213a44d96f5  |Wed, 24 Apr 2024 19:28:46 GMT  |
|models/vect_review.joblib.gz    |27852416  |92d019222e89c2db309379b0584dc958  |Wed, 24 Apr 2024 15:51:49 GMT  |
|models/vect_review1.joblib.gz   |27852432  |72b6c79a63956493b3b75f1d90c9de0a  |Wed, 24 Apr 2024 18:08:25 GMT  |
|models/vect_review2.joblib.gz   |27852448  |e21ad0cbc5fa7aff688a02d969d73dc0  |Wed, 24 Apr 2024